In [12]:
!pip install imblearn lightgbm optuna

In [13]:
import pandas as pd
import numpy as np
import joblib
import os
import warnings
import time

# --- 引入模型與工具 ---
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, f1_score,
    roc_auc_score, classification_report
)
from imblearn.over_sampling import SMOTE 

# --- 環境設定 ---
warnings.filterwarnings('ignore')

In [16]:
# 🎯 定義儲存模型人工製品的路徑，並確保目錄存在
SCALER_SAVE_PATH = "./model_artifacts" 
if not os.path.exists(SCALER_SAVE_PATH):
    os.makedirs(SCALER_SAVE_PATH, exist_ok=True)
    print(f"✅ 目錄已創建: {SCALER_SAVE_PATH}")


FILE_PATHS = {
    # 請確保這些 CSV 檔案的特徵數量與模型訓練時的 60 個特徵一致
    'X_train_resampled': 'processed_data/X_train_resampled.csv',
    'y_train_resampled': 'processed_data/y_train_resampled.csv',
    'X_test': 'processed_data/X_test.csv',
    'y_test': 'processed_data/y_test.csv'
}
print("--- 1. 載入前處理後的資料 ---")
try:
    # 載入平衡後的訓練集
    X_train = pd.read_csv(FILE_PATHS['X_train_resampled'])
    y_train = pd.read_csv(FILE_PATHS['y_train_resampled']).squeeze()
    
    # 載入測試集
    X_test = pd.read_csv(FILE_PATHS['X_test'])
    y_test = pd.read_csv(FILE_PATHS['y_test']).squeeze()
    
    print(f"✅ 資料載入成功。")
    print(f"訓練集 (SMOTE後) 原始特徵數: {X_train.shape[1]}，筆數: {len(X_train)}")
    print(f"測試集 原始特徵數: {X_test.shape[1]}，筆數: {len(X_test)}")
    
except FileNotFoundError as e:
    print(f"⚠️ 錯誤：找不到檔案 {e}。請確認 'processed_data' 資料夾和檔案路徑是否正確。")
    exit()

--- 1. 載入前處理後的資料 ---
✅ 資料載入成功。
訓練集 (SMOTE後) 原始特徵數: 123，筆數: 101280
測試集 原始特徵數: 123，筆數: 16266


In [17]:
# 步驟 2: 建立最終訓練集與獨立驗證集 (定義 X_train_final)
# ==============================================================================
print("\n--- 2. 資料切分 (訓練集 -> 最終訓練集 + 獨立驗證集) ---")

# 從 SMOTE 後的訓練集中切分 10% 作為獨立的驗證集
X_train_final, X_val, y_train_final, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

print(f"用於 GridSearchCV 的最終訓練集筆數: {len(X_train_final)}")
print(f"獨立驗證集筆數: {len(X_val)}")
print(f"獨立測試集筆數: {len(X_test)}")


--- 2. 資料切分 (訓練集 -> 最終訓練集 + 獨立驗證集) ---
用於 GridSearchCV 的最終訓練集筆數: 91152
獨立驗證集筆數: 10128
獨立測試集筆數: 16266


In [18]:
# 步驟 3: 標準化 (Scaling) 與特徵對齊修正 (避免 ValueError)
# ==============================================================================
print("\n--- 3. 標準化 (Scaling) 與特徵對齊修正 ---")

# **🎯 核心邏輯：確保 X_test 欄位與 X_train_final 欄位數量和順序完全一致 **
train_cols = X_train_final.columns.tolist()
original_test_cols = X_test.shape[1]

if original_test_cols != len(train_cols):
    print(f"⚠️ 警告：測試集特徵數 ({original_test_cols}) 與訓練集特徵數 ({len(train_cols)}) 不一致。執行對齊...")
    
    # 執行特徵對齊：新增訓練集有但測試集沒有的欄位，並刪除測試集多餘的欄位
    X_test = X_test.reindex(columns=train_cols, fill_value=0)
    
    if X_test.shape[1] == len(train_cols):
         print(f"✅ X_test 特徵數量已修正並對齊。修正後特徵數: {X_test.shape[1]}")
    else:
         # 如果到這一步還不對齊，那說明原始資料集有嚴重問題
         print(f"❌ 錯誤：特徵對齊失敗，請檢查原始 CSV 檔案的內容。")
         exit() # 如果無法對齊，則終止程式避免錯誤的訓練
         
# 建立 Scaler
scaler = StandardScaler()

# 僅使用最終訓練集 X_train_final 來 fit Scaler
X_train_final_scaled = pd.DataFrame(scaler.fit_transform(X_train_final), columns=train_cols)

# 使用 fit 好的 Scaler 來 transform 驗證集和測試集
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=train_cols)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=train_cols) # 使用修正後的 X_test

# 儲存重要工具 (Scaler 和 欄位名稱)
joblib.dump(scaler, os.path.join(SCALER_SAVE_PATH, "scaler.pkl"))
joblib.dump(train_cols, os.path.join(SCALER_SAVE_PATH, "feature_names.pkl"))
print(f"✅ Scaler 與特徵名稱 (共 {len(train_cols)} 個) 已儲存到 {SCALER_SAVE_PATH}。")



--- 3. 標準化 (Scaling) 與特徵對齊修正 ---
✅ Scaler 與特徵名稱 (共 123 個) 已儲存到 ./model_artifacts。


In [19]:
import pandas as pd
import numpy as np
import joblib
import os
import warnings
import time
from sklearn.metrics import f1_score, fbeta_score # 🎯 引入 fbeta_score (用於 F2-Score)
from tqdm import tqdm # 引入 tqdm 增加進度條

# --- 引入模型與工具 ---
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import (
    accuracy_score, confusion_matrix, f1_score, fbeta_score, # 確保 fbeta_score 被引入
    roc_auc_score, classification_report, make_scorer
)

# ==============================================================================
# 步驟 4: 模型訓練與超參數調校 (GridSearchCV) - 優先優化召回率 (F2-Score)
# ==============================================================================
print("\n--- 4. 模型訓練與 GridSearchCV 調校 (使用 F2-Score 優先優化召回率) ---")

rf_model = RandomForestClassifier(random_state=42)

# 🎯 進一步調整：擴大 min_samples_leaf 範圍，嘗試更多泛化能力組合
param_grid = {
    'n_estimators': [150, 300],          
    'max_depth': [25, 30, 35],           
    'min_samples_leaf': [1, 5, 10],      
    'criterion': ['gini', 'entropy']     
}

# 🎯 核心變更：使用 F2-Score (beta=2) 作為優化目標。
# F2-Score 賦予召回率 (Recall) 更高的權重，目標是減少漏報 (FN)，讓模型更能成功捕捉下雨事件。
f2_scorer = make_scorer(fbeta_score, pos_label=1, beta=2)

start_time = time.time()
grid_search = GridSearchCV(
    estimator=rf_model, 
    param_grid=param_grid, 
    scoring=f2_scorer,      # *** 更改：使用 F2-score (針對 Rain 類別) 作為主要優化目標 ***
    cv=3,                   
    verbose=1,
    n_jobs=-1
)
# 使用平衡且標準化後的最終訓練集進行訓練與交叉驗證
grid_search.fit(X_train_final_scaled, y_train_final)
end_time = time.time()

best_rf_model = grid_search.best_estimator_
print("\n--- 訓練與調校結果彙報 ---")
print(f"🥇 最佳超參數組合: {grid_search.best_params_}")
print(f"總訓練與調校時間: {end_time - start_time:.2f} 秒")


# ==============================================================================
# 步驟 5: 驗證步驟：模型於獨立驗證集上評估 (預設閾值 0.5)
# ==============================================================================
print("\n--- 5. 驗證步驟：模型於獨立驗證集上評估 (預設閾值 0.5) ---")
# 使用最佳模型對驗證集進行預測 (使用標準化後的 X_val_scaled)
y_val_pred_default = best_rf_model.predict(X_val_scaled)
y_val_proba = best_rf_model.predict_proba(X_val_scaled)[:, 1]

val_accuracy = accuracy_score(y_val, y_val_pred_default)
val_f1 = f1_score(y_val, y_val_pred_default)
val_auc = roc_auc_score(y_val, y_val_proba)

print(f"🔍 驗證集 準確率 (Accuracy): {val_accuracy:.4f}")
print(f"🔍 驗證集 F1-Score: {val_f1:.4f}")
print(f"🔍 驗證集 AUC Score: {val_auc:.4f}")


# ==============================================================================
# 步驟 6: 決策閾值優化 (Threshold Optimization) - 調整步長
# ==============================================================================
print("\n--- 6. 決策閾值優化：在驗證集上尋找最佳閾值 (最大化 Rain 類別 F1-Score) ---")

best_f1_rain = 0
optimal_threshold = 0.5

# 🎯 調整搜索步長為 0.005，尋找更精確的閾值
thresholds = np.arange(0.10, 0.91, 0.005)

# 使用 tqdm 顯示進度條
for threshold in tqdm(thresholds, desc="尋找最佳閾值"):
    # 使用當前閾值進行預測
    y_val_pred_thresh = (y_val_proba >= threshold).astype(int)
    
    # 計算「Rain (1)」類別的 F1-Score (這裡仍使用 F1 作為最終部署的平衡指標)
    f1_rain_class = f1_score(y_val, y_val_pred_thresh, pos_label=1)
    
    if f1_rain_class > best_f1_rain:
        best_f1_rain = f1_rain_class
        optimal_threshold = threshold

print(f"\n🥇 最佳 F1-Score (Rain 類別) 是 {best_f1_rain:.4f}，對應的最佳閾值是 {optimal_threshold:.3f}")


# ==============================================================================
# 步驟 7: 最終模型效能評估 (使用最佳閾值於測試集)
# ==============================================================================
print("\n--- 7. 最終評估：模型於測試集上評估 (使用最佳閾值) ---")
# 使用測試集上的概率
y_test_proba = best_rf_model.predict_proba(X_test_scaled)[:, 1]

# 根據優化後的閾值進行最終預測
y_test_pred_optimized = (y_test_proba >= optimal_threshold).astype(int)

# 計算指標
accuracy = accuracy_score(y_test, y_test_pred_optimized)
f1 = f1_score(y_test, y_test_pred_optimized, average='weighted') # 使用加權平均F1作為總體指標
cm = confusion_matrix(y_test, y_test_pred_optimized)
class_report = classification_report(y_test, y_test_pred_optimized, target_names=['No Rain (0)', 'Rain (1)'])

print(f"✅ 準確率 (Accuracy): {accuracy:.4f}")
print(f"✅ 加權 F1-Score: {f1:.4f}")
print(f"✅ AUC Score (閾值不影響): {roc_auc_score(y_test, y_test_proba):.4f}")

print("\n--- 混淆矩陣 (Confusion Matrix) ---")
print(f"[[TN, FP] (TN=不下雨正確, FP=誤報下雨)\n [FN, TP]] (FN=漏報下雨, TP=下雨正確)")
print(cm)

print("\n--- 分類報告 (Classification Report) ---")
print(class_report)



--- 4. 模型訓練與 GridSearchCV 調校 (使用 F2-Score 優先優化召回率) ---
Fitting 3 folds for each of 36 candidates, totalling 108 fits

--- 訓練與調校結果彙報 ---
🥇 最佳超參數組合: {'criterion': 'entropy', 'max_depth': 30, 'min_samples_leaf': 1, 'n_estimators': 300}
總訓練與調校時間: 601.28 秒

--- 5. 驗證步驟：模型於獨立驗證集上評估 (預設閾值 0.5) ---
🔍 驗證集 準確率 (Accuracy): 0.9109
🔍 驗證集 F1-Score: 0.9111
🔍 驗證集 AUC Score: 0.9738

--- 6. 決策閾值優化：在驗證集上尋找最佳閾值 (最大化 Rain 類別 F1-Score) ---


尋找最佳閾值: 100%|██████████| 162/162 [00:00<00:00, 331.06it/s]



🥇 最佳 F1-Score (Rain 類別) 是 0.9117，對應的最佳閾值是 0.515

--- 7. 最終評估：模型於測試集上評估 (使用最佳閾值) ---
✅ 準確率 (Accuracy): 0.8543
✅ 加權 F1-Score: 0.8517
✅ AUC Score (閾值不影響): 0.8900

--- 混淆矩陣 (Confusion Matrix) ---
[[TN, FP] (TN=不下雨正確, FP=誤報下雨)
 [FN, TP]] (FN=漏報下雨, TP=下雨正確)
[[11641  1019]
 [ 1351  2255]]

--- 分類報告 (Classification Report) ---
              precision    recall  f1-score   support

 No Rain (0)       0.90      0.92      0.91     12660
    Rain (1)       0.69      0.63      0.66      3606

    accuracy                           0.85     16266
   macro avg       0.79      0.77      0.78     16266
weighted avg       0.85      0.85      0.85     16266



In [20]:
# 步驟 7: 儲存最終模型 (供部署使用)
# ==============================================================================
print("\n--- 7. 模型儲存 ---")

# 使用 joblib 儲存最佳模型 (副檔名 .pth 是根據您的要求設定)
model_filename = 'random_forest_final_model.pth' 
joblib.dump(best_rf_model, model_filename)

print(f"✨ 最佳隨機森林模型已儲存至：{model_filename}")
print(f"✨ StandardScaler 與特徵名稱已儲存至：{SCALER_SAVE_PATH} 目錄")


--- 7. 模型儲存 ---
✨ 最佳隨機森林模型已儲存至：random_forest_final_model.pth
✨ StandardScaler 與特徵名稱已儲存至：./model_artifacts 目錄


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
import sys
import warnings

# 忽略警告
warnings.filterwarnings('ignore')

# --- 模型檔案路徑設定 ---
MODEL_PATH = 'random_forest_final_model.pth' 
SCALER_PATH = './model_artifacts/scaler.pkl'
FEATURES_PATH = './model_artifacts/feature_names.pkl'
# 🥇 最佳決策閾值 (根據 F2 優化後的結果 0.515)
OPTIMAL_THRESHOLD = 0.515 

# --- 特徵定義 (與訓練模型保持一致) ---
NUMERICAL_FEATURES = [
    'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 
    'WindGustSpeed', 'WindSpeed9am', 'Humidity9am'
]
CATEGORICAL_FEATURES = [
    'Location', 'WindDir9am', 'Month'
]
CATEGORY_CHOICES = {
    'Location': ['Sydney', 'Melbourne', 'Perth', 'Darwin', 'Canberra', 'Brisbane', 'Adelaide', 'Hobart', 'Albury', 'WaggaWagga', '其他'], 
    'WindDir9am': ['N', 'NE', 'E', 'SE', 'S', 'SW', 'W', 'NW', '其他'], 
    'Month': [str(i) for i in range(1, 13)]
}

# --- 載入模型與工具 ---
try:
    # 載入模型
    model = joblib.load(MODEL_PATH)
    # 載入 StandardScaler
    scaler = joblib.load(SCALER_PATH)
    # 載入訓練時使用的特徵名稱列表 (包含所有數值和 One-Hot 特徵)
    model_features = joblib.load(FEATURES_PATH)
    
    print("✅ 模型與工具載入成功！")
    print(f"模型預期 {len(model_features)} 個特徵。")

except Exception as e:
    print(f"❌ 載入模型或工具時發生錯誤: {e}")
    print("請確保 'random_forest_final_model.pth' 和 'model_artifacts/' 資料夾存在且包含必要的檔案。")
    sys.exit(1) # 如果載入失敗則終止程式

# ==============================================================================
# 函數: 獲取用戶輸入 (CLI 交互)
# ==============================================================================
def get_user_input():
    """交互式獲取用戶輸入。"""
    print("\n" + "="*40)
    print("  請輸入氣象數據進行降雨預測  ")
    print("="*40)
    raw_inputs = []

    # 1. 數值特徵輸入
    print("\n--- 數值特徵輸入 ---")
    for feature in NUMERICAL_FEATURES:
        while True:
            try:
                # 提示用戶輸入
                user_input = input(f"請輸入 {feature} (數值): ").strip()
                if not user_input:
                     # 允許空輸入並使用 Pandas 預設的 NaN 處理 (但由於我們初始化為 0，這裡必須要求輸入)
                     raise ValueError("輸入不能為空。")
                     
                value = float(user_input)
                raw_inputs.append(value)
                break
            except ValueError:
                print("輸入錯誤：請輸入有效的數字。")

    # 2. 分類特徵輸入
    print("\n--- 分類特徵輸入 ---")
    for feature in CATEGORICAL_FEATURES:
        choices = CATEGORY_CHOICES.get(feature, [])
        while True:
            if feature == 'Location':
                print(f"地點選項範例: {', '.join(choices[:5])}...")
            elif feature == 'WindDir9am':
                print(f"風向選項範例: {', '.join(choices[:8])}")
            elif feature == 'Month':
                print(f"月份選項: {', '.join(choices)}")
            
            user_input = input(f"請輸入 {feature}: ").strip()
            
            if user_input:
                # 針對 Month，確保是 1-12 的整數
                if feature == 'Month':
                    try:
                        month_int = int(user_input)
                        if 1 <= month_int <= 12:
                            raw_inputs.append(month_int)
                            break
                        else:
                            print("輸入錯誤：月份必須在 1 到 12 之間。")
                    except ValueError:
                        print("輸入錯誤：月份必須是整數。")
                else:
                    raw_inputs.append(user_input)
                    break
            else:
                print("輸入不能為空，請重新輸入。")
                
    return raw_inputs

# ==============================================================================
# 函數: 執行預測並輸出結果
# ==============================================================================
def predict_cli(raw_inputs):
    """將原始輸入轉換為模型格式，執行預測，並輸出結果。"""
    
    # 1. 根據 model_features 列表，構造一個空的 DataFrame
    input_df = pd.DataFrame(0.0, index=[0], columns=model_features)

    # 2. 填充數值特徵
    num_inputs = raw_inputs[:len(NUMERICAL_FEATURES)]
    for feature, value in zip(NUMERICAL_FEATURES, num_inputs):
        # 僅填充模型中實際存在的特徵
        if feature in input_df.columns:
            input_df[feature].iloc[0] = value
            
    # 3. 填充分類特徵 (One-Hot Encoding 轉換)
    cat_inputs = raw_inputs[len(NUMERICAL_FEATURES):]
    
    # Location
    location_input = cat_inputs[0]
    ohe_col_location = f'Location_{location_input}'
    if ohe_col_location in input_df.columns:
        input_df[ohe_col_location].iloc[0] = 1.0
    
    # WindDir9am
    winddir9am_input = cat_inputs[1]
    ohe_col_winddir = f'WindDir9am_{winddir9am_input}'
    if ohe_col_winddir in input_df.columns:
        input_df[ohe_col_winddir].iloc[0] = 1.0
    
    # Month
    month_input = cat_inputs[2]
    ohe_col_month = f'Month_{int(month_input)}'
    if ohe_col_month in input_df.columns:
        input_df[ohe_col_month].iloc[0] = 1.0
        
    # 4. 標準化 (Scaling)
    input_scaled = pd.DataFrame(scaler.transform(input_df), columns=model_features)

    # 5. 進行預測
    proba = model.predict_proba(input_scaled)[:, 1][0]
    
    # 🎯 使用最佳閾值進行最終決策
    prediction = 1 if proba >= OPTIMAL_THRESHOLD else 0
    
    # 6. 格式化輸出
    prediction_label = "✅ 是 (預測會下雨)" if prediction == 1 else "❌ 否 (預測不會下雨)"
    confidence = f"{proba * 100:.2f}%"

    print("\n" + "="*40)
    print("        ✨ 預測結果 ✨       ")
    print("="*40)
    print(f"預測機率 (明日下雨, P=1)： {confidence}")
    print(f"最終判斷 (閾值 {OPTIMAL_THRESHOLD}): {prediction_label}")
    print("="*40)

# ==============================================================================
# 主程式執行區塊
# ==============================================================================
if __name__ == "__main__":
    while True:
        try:
            user_inputs = get_user_input()
            predict_cli(user_inputs)
        except Exception as e:
            print(f"\n發生嚴重錯誤: {e}")
            print("請檢查您的輸入格式是否正確。")
        
        # 詢問用戶是否繼續
        print("\n是否要進行新的預測？ (輸入 'y' 繼續，其他任意鍵結束)")
        continue_query = input().strip().lower()
        if continue_query != 'y':
            break

    print("\n預測程式結束。感謝您的使用！")

✅ 模型與工具載入成功！
模型預期 123 個特徵。

  請輸入氣象數據進行降雨預測  

--- 數值特徵輸入 ---
